## Depedencies

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path



## Data Analysis & Manipulation

### Census Data

In [2]:
census_data_path = str(Path.cwd().joinpath('calls/census_data/census_data.csv'))

In [3]:
census_df = pd.read_csv(census_data_path)

In [4]:
census_df.head()

,GEO.id,GEO.id2,GEO.display-label,D001
0,Id,Id2,Geography,Total
1,8600000US06390,06390,ZCTA5 06390,236
2,8600000US10001,10001,ZCTA5 10001,21102
3,8600000US10002,10002,ZCTA5 10002,81410
4,8600000US10003,10003,ZCTA5 10003,56024


### Database Connection

In [31]:
db_path = str(Path.cwd().joinpath('calls/app.db'))
conn = sqlite3.connect(db_path)
# cur = conn.cursor()

In [34]:
db = pd.read_sql_query('select * from locations', conn, index_col = "id")

In [35]:
db.zip = db.zip.astype('float', inplace = True)
db.zip = db.zip.astype('int')


In [36]:
db.head()

,zip,borough_id
id,,
1,10027,1
2,11223,2
3,10461,3
4,10002,1
5,11385,4


In [ ]:
# cur.close()
conn.close()

### Reading in data
Data read in and non-relevent columns dropped (same as from the original)

In [8]:
raw_data = pd.read_csv('calls/everything.csv', index_col=0, dtype={'incident_zip': str})

/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
raw_data.drop(columns = ['Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Park Facility Name', 'Park Borough', 'Vehicle Type',
       'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment',
       'Latitude', 'Longitude', 'Status', 'Facility Type', 'Due Date', 'Resolution Description',
       'Resolution Action Updated Date', 'Community Board', 'BBL',], inplace=True)

In [10]:
raw_data.columns

Index(['Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Borough', 'X Coordinate (State Plane)', 'Y Coordinate (State Plane)',
       'Open Data Channel Type', 'Location'],
      dtype='object')

In [11]:
raw_data.head(1)

,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Location
Unique Key,,,,,,,,,,,,,
38490120,02/17/2018 10:03:19 AM,02/17/2018 10:42:22 AM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10027.0,MANHATTAN,995846.0,236544.0,ONLINE,"(40.81592674227958, -73.95810673319008)"


Fixing the incident zip datatype after filling in NaN values

In [12]:
raw_data.fillna("00000", inplace=True)
raw_data['Incident Zip'] = raw_data['Incident Zip'].astype('float')
raw_data['Incident Zip'] = raw_data['Incident Zip'].astype('int')
raw_data.head(1)

,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Location
Unique Key,,,,,,,,,,,,,
38490120,02/17/2018 10:03:19 AM,02/17/2018 10:42:22 AM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10027,MANHATTAN,995846,236544,ONLINE,"(40.81592674227958, -73.95810673319008)"


Creating Population column

In [13]:
raw_data.loc[:,'Population'] = pd.Series(0, index=raw_data.index)
raw_data.head(3)

,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Location,Population
Unique Key,,,,,,,,,,,,,,
38490120,02/17/2018 10:03:19 AM,02/17/2018 10:42:22 AM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10027,MANHATTAN,995846,236544,ONLINE,"(40.81592674227958, -73.95810673319008)",0
38490121,02/17/2018 08:40:26 AM,02/17/2018 03:34:25 PM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11223,BROOKLYN,990992,156670,ONLINE,"(40.59669663536788, -73.97572291796888)",0
38490122,02/17/2018 08:32:52 PM,02/17/2018 11:35:48 PM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10461,BRONX,1.03131e+06,249965,PHONE,"(40.8526457301192, -73.82990522576051)",0


In [14]:
census_df.drop(0, inplace=True)

In [15]:
len(census_df)

1794

Mapping population information to zip<br><br>
GEO.id2 is zipcode in the census_df

In [16]:
census_df.columns

Index(['GEO.id', 'GEO.id2', 'GEO.display-label', 'D001'], dtype='object')

In [17]:
pop_dict = {}

for index, row in census_df.iterrows():    
    pop_dict[row['GEO.id2']] = pop_dict.get(row['GEO.id2'], row.D001)

In [18]:
raw_data['Incident Zip'] = raw_data['Incident Zip'].astype('str')
raw_data['Population'] = raw_data['Incident Zip'].map(pop_dict)
raw_data.head(3)

,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Location,Population
Unique Key,,,,,,,,,,,,,,
38490120,02/17/2018 10:03:19 AM,02/17/2018 10:42:22 AM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10027,MANHATTAN,995846,236544,ONLINE,"(40.81592674227958, -73.95810673319008)",59707
38490121,02/17/2018 08:40:26 AM,02/17/2018 03:34:25 PM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11223,BROOKLYN,990992,156670,ONLINE,"(40.59669663536788, -73.97572291796888)",78731
38490122,02/17/2018 08:32:52 PM,02/17/2018 11:35:48 PM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10461,BRONX,1.03131e+06,249965,PHONE,"(40.8526457301192, -73.82990522576051)",50502


Checked to ensure correct mapping

In [19]:
print(pop_dict['10027'])
print(pop_dict['11223'])
print(pop_dict['10461'])

59707
78731
50502


In [20]:
raw_data.dtypes

Created Date                  object
Closed Date                   object
Agency                        object
Agency Name                   object
Complaint Type                object
Descriptor                    object
Location Type                 object
Incident Zip                  object
Borough                       object
X Coordinate (State Plane)    object
Y Coordinate (State Plane)    object
Open Data Channel Type        object
Location                      object
Population                    object
dtype: object

Now mapping neighborhoods onto zipcode as well

In [21]:
hood_path = str(Path.cwd().joinpath('calls/neighborhood.csv'))
hoods = pd.read_csv(hood_path, index_col=0)
hoods_dict = hoods.to_dict(orient='index')
for k, v in hoods_dict.items():
    hoods_dict[k] = hoods_dict[k]['Neighborhood']

In [22]:
# hoods.head()

In [23]:
hoods_dict

{10001: 'Chelsea and Clinton',
 10002: 'Lower East Side',
 10003: 'Lower East Side',
 10004: 'Lower Manhattan',
 10005: 'Lower Manhattan',
 10006: 'Lower Manhattan',
 10007: 'Lower Manhattan',
 10009: 'Lower East Side',
 10010: 'Gramercy Park and Murray Hill',
 10011: 'Chelsea and Clinton',
 10012: 'Greenwich Village and Soho',
 10013: 'Greenwich Village and Soho',
 10014: 'Greenwich Village and Soho',
 10016: 'Gramercy Park and Murray Hill',
 10017: 'Gramercy Park and Murray Hill',
 10018: 'Chelsea and Clinton',
 10019: 'Chelsea and Clinton',
 10020: 'Chelsea and Clinton',
 10021: 'Upper East Side',
 10022: 'Gramercy Park and Murray Hill',
 10023: 'Upper West Side',
 10024: 'Upper West Side',
 10025: 'Upper West Side',
 10026: 'Central Harlem',
 10027: 'Central Harlem',
 10028: 'Upper East Side',
 10029: 'East Harlem',
 10030: 'Central Harlem',
 10031: 'Inwood and Washington Heights',
 10032: 'Inwood and Washington Heights',
 10033: 'Inwood and Washington Heights',
 10034: 'Inwood and

In [24]:
raw_data['Incident Zip'] = raw_data['Incident Zip'].astype('int')
raw_data['Neighborhood'] = raw_data['Incident Zip'].map(hoods_dict)

In [25]:
raw_data.head()

,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Location,Population,Neighborhood
Unique Key,,,,,,,,,,,,,,,
38490120,02/17/2018 10:03:19 AM,02/17/2018 10:42:22 AM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10027,MANHATTAN,995846,236544,ONLINE,"(40.81592674227958, -73.95810673319008)",59707,Central Harlem
38490121,02/17/2018 08:40:26 AM,02/17/2018 03:34:25 PM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11223,BROOKLYN,990992,156670,ONLINE,"(40.59669663536788, -73.97572291796888)",78731,Southern Brooklyn
38490122,02/17/2018 08:32:52 PM,02/17/2018 11:35:48 PM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10461,BRONX,1.03131e+06,249965,PHONE,"(40.8526457301192, -73.82990522576051)",50502,Southeast Bronx
38490123,02/17/2018 02:07:29 PM,02/17/2018 02:21:28 PM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10002,MANHATTAN,987058,198677,PHONE,"(40.71199877267997, -73.98987125976987)",81410,Lower East Side
38490124,02/17/2018 10:52:04 PM,02/18/2018 01:30:55 AM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11385,QUEENS,1.01212e+06,191989,PHONE,"(40.693598357245065, -73.89951223535249)",98592,West Central Queens


For each zip code (and for each neighborhood) 

Calls/person 
Calls of each type/person 



### Creating the neighborhoods information
Scrapped a website to get the zips by neighborhood info using requests and BeautifulSoup; did some manipulation

In [119]:
# import urllib.request, urllib.parse, urllib.error
# from bs4 import BeautifulSoup
# import ssl
# import pandas as pd

# ctx = ssl.create_default_context()
# ctx.check_hostname = False
# ctx.verify_mode = ssl.CERT_NONE

# url = input('Enter - ')
# if len(url) < 1: url = 'https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm'

# soup = BeautifulSoup(urllib.request.urlopen(url, context=ctx).read(), 'html.parser')

# tags = soup('td')

# #Pass tags to this function. It will pull out each neighborhood and each set of zip codes,
# #and then zip those together into a dictionary with neighborhoods paired to their zipcodes

# def create_hood_dict(tag_list):
#     neighborhood = []
#     zip_codes = []
#     for tag in tag_list:
#         if str(tag).startswith('<td headers="header2">'):
#             hood = tag.text.lstrip()
#             neighborhood.append(hood)
#         elif str(tag).startswith('<td headers="header3">'):
#             zips_1 = tag.text.split(",")
#             zips_2 = [zip.lstrip() for zip in zips_1]
#             zip_codes.append(zips_2)
#         else:
#             continue
#     result = zip(neighborhood, zip_codes)
#     neighborhoods_local = dict(result)
#     return neighborhoods_local


# def persist(dict_name):
#     better_dict = {}
    
#     for k, v in dict_name.items():
#         for zipcode in v: 
#             better_dict[zipcode] = better_dict.get(zipcode, k)
            
#     data = test3 = pd.DataFrame.from_dict(better_dict, orient = 'index', columns = ['Neighborhood'])
#     data.to_csv('calls/neighborhood.csv', mode = 'w+')
#     return data

Enter - 


In [120]:
hood_dict = create_hood_dict(tags)
test1 = persist(hood_dict)

In [101]:
hood_dict

{'Borough Park': ['11204.0', '11218.0', '11219.0', '11230.0'],
 'Bronx Park and Fordham': ['10458.0', '10467.0', '10468.0'],
 'Bushwick and Williamsburg': ['11206.0', '11221.0', '11237.0'],
 'Canarsie and Flatlands': ['11234.0', '11236.0', '11239.0'],
 'Central Bronx': ['10453.0', '10457.0', '10460.0'],
 'Central Brooklyn': ['11212.0', '11213.0', '11216.0', '11233.0', '11238.0'],
 'Central Harlem': ['10026.0', '10027.0', '10030.0', '10037.0', '10039.0'],
 'Central Queens': ['11365.0', '11366.0', '11367.0'],
 'Chelsea and Clinton': ['10001.0',
  '10011.0',
  '10018.0',
  '10019.0',
  '10020.0',
  '10036.0'],
 'East Harlem': ['10029.0', '10035.0'],
 'East New York and New Lots': ['11207.0', '11208.0'],
 'Flatbush': ['11203.0', '11210.0', '11225.0', '11226.0'],
 'Gramercy Park and Murray Hill': ['10010.0', '10016.0', '10017.0', '10022.0'],
 'Greenpoint': ['11211.0', '11222.0'],
 'Greenwich Village and Soho': ['10012.0', '10013.0', '10014.0'],
 'High Bridge and Morrisania': ['10451.0', '10

In [121]:
test1

,Neighborhood
11101,Northwest Queens
10018,Chelsea and Clinton
10025,Upper West Side
10466,Northeast Bronx
10301,Stapleton and St. George
11411,Southeast Queens
10033,Inwood and Washington Heights
10006,Lower Manhattan
11212,Central Brooklyn
11691,Rockaways


In [158]:
raw_data.head(3)

,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Location,Population,Neighborhood
Unique Key,,,,,,,,,,,,,,,
38490120,02/17/2018 10:03:19 AM,02/17/2018 10:42:22 AM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10027,MANHATTAN,995846,236544,ONLINE,"(40.81592674227958, -73.95810673319008)",59707,Central Harlem
38490121,02/17/2018 08:40:26 AM,02/17/2018 03:34:25 PM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11223,BROOKLYN,990992,156670,ONLINE,"(40.59669663536788, -73.97572291796888)",78731,Southern Brooklyn
38490122,02/17/2018 08:32:52 PM,02/17/2018 11:35:48 PM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10461,BRONX,1.03131e+06,249965,PHONE,"(40.8526457301192, -73.82990522576051)",50502,Southeast Bronx


In [156]:
# raw_dict = raw_data.to_dict(orient='index')

In [26]:
def total_complaints():
    total_complaints_dict = {}
    for i in list(set(raw_data['Incident Zip'])):
        total_complaints_dict[i] = total_complaints_dict.get(i, len(raw_data.loc[raw_data['Incident Zip'] == i]))
    return total_complaints_dict

total_complaints_dict = total_complaints()
total_complaints_dict

{0: 2791,
 83: 36,
 10000: 162,
 10001: 3091,
 10002: 10948,
 10003: 6572,
 10004: 229,
 10005: 405,
 10006: 172,
 10007: 818,
 10009: 9227,
 10010: 1939,
 10011: 4045,
 10012: 4251,
 10013: 2674,
 10014: 3471,
 10016: 3476,
 10017: 1010,
 10018: 1016,
 10019: 2855,
 10020: 25,
 10021: 1731,
 10022: 1450,
 10023: 2517,
 10024: 3095,
 10025: 6637,
 10026: 7152,
 10027: 7672,
 10028: 2501,
 10029: 7590,
 10030: 4500,
 10031: 15036,
 10032: 10794,
 10033: 10117,
 10034: 10215,
 10035: 4279,
 10036: 3783,
 10037: 2105,
 10038: 1248,
 10039: 4713,
 10040: 12228,
 10041: 11,
 10044: 48,
 10048: 1,
 10065: 1286,
 10069: 58,
 10075: 1256,
 10103: 2,
 10106: 2,
 10107: 3,
 10110: 3,
 10111: 1,
 10112: 5,
 10118: 18,
 10119: 12,
 10120: 9,
 10121: 38,
 10122: 3,
 10123: 1,
 10128: 2658,
 10151: 1,
 10153: 1,
 10155: 1,
 10162: 9,
 10165: 1,
 10168: 1,
 10169: 1,
 10171: 1,
 10172: 1,
 10173: 3,
 10175: 1,
 10177: 3,
 10178: 1,
 10271: 1,
 10278: 12,
 10279: 1,
 10280: 121,
 10281: 17,
 10282: 31

In [182]:
raw_data['Descriptor'] = raw_data['Descriptor'].astype('str')

def specific_complaint_type_count(complaint_type):
    specific_complaints_dict = {}
    for i in list(set(raw_data['Incident Zip'])):
        specific_complaints_dict[i] = specific_complaints_dict.get(i, len(raw_data.loc([raw_data['Incident Zip'] == i]
                                                                    & raw_data.loc[raw_data['Descriptor'] == complaint_type])))
    return specific_complaints_dict

specific_complaints_dict = specific_complaint_type_count('Loud Music/Party')
specific_complaints_dict

ValueError: Unable to coerce to Series, length must be 15: given 1

In [ ]:
# now call them, and build out a dataframe, then get your 'max', 'min' 'avg' and per capita of those 

In [170]:


len(list(set(raw_data['Incident Zip'])))

222

In [ ]:
& df['other_column'].isin(some_values)]

## Updating SQL Database

Notes on creating and populating a new column, courtesy of StackOverflow

ALTER TABLE table ADD COLUMN col3 desired_datatype;

SELECT a, b, (case when (a + b) % 2 = 1 then 'Y' else 'N' end) AS col3 FROM table t;

OR  

UPDATE table SET col3 = (case when (a + b) % 2 = 1 then 'Y' else 'N' end) ;


## The Important Code

In [51]:
test_44 = pd.read_sql("SELECT * FROM locations", conn, index_col = "id")

In [53]:
#Fixing Datatype 

test_44['zip'] = test_44['zip'].astype('float')
test_44['zip'] = test_44['zip'].astype('int')

#Population 

test_44['zip']= test_44['zip'].astype('str')
test_44['population'] = test_44['zip'].map(pop_dict)

#Neighborhoods 

test_44['zip'] = test_44['zip'].astype('int')
test_44['neighborhood'] = test_44['zip'].map(hoods_dict)

In [57]:
test_44.isna().sum()

zip              0
borough_id       0
population      21
neighborhood    46
dtype: int64

In [59]:
# I was getting an error about NaN values, apparently some of these zipcodes don't belong to any neighborhood
# or a listed population 

test_44.fillna({'population':0, 'neighborhood':'Undefined'}, inplace=True)
test_44['neighborhood'] = test_44['neighborhood'].astype('str')
test_44['population'] = test_44['population'].astype('int')

In [60]:
test_44.head()

,zip,borough_id,population,neighborhood
id,,,,
1,10027,1,59707,Central Harlem
2,11223,2,78731,Southern Brooklyn
3,10461,3,50502,Southeast Bronx
4,10002,1,81410,Lower East Side
5,11385,4,98592,West Central Queens


In [61]:
test_44.to_sql("locations", conn, if_exists = 'replace', index_label = "id")

In [62]:
test_write_success = pd.read_sql("SELECT * FROM locations", conn, index_col = "id")
test_write_success.head()

,zip,borough_id,population,neighborhood
id,,,,
1,10027,1,59707,Central Harlem
2,11223,2,78731,Southern Brooklyn
3,10461,3,50502,Southeast Bronx
4,10002,1,81410,Lower East Side
5,11385,4,98592,West Central Queens


In [63]:
test_write_success.shape

(222, 4)

In [64]:
test_write_success.dtypes

zip              int64
borough_id       int64
population       int64
neighborhood    object
dtype: object

## Comparison Metrics

In [68]:
test_write_success.groupby('borough_id')['population'].sum()

borough_id
1    1577374
2    2504700
3    1382480
4    2311269
5     468730
6      38982
Name: population, dtype: int64

In [76]:
neighborhood_pops = dict(test_write_success.groupby('neighborhood')['population'].sum())
# could also use list here!

In [94]:
test_write_success.groupby('zip')['population'].sum()

zip
0            0
83           0
10000        0
10001    21102
10002    81410
10003    56024
10004     3089
10005     7135
10006     3011
10007     6988
10009    61347
10010    31834
10011    50984
10012    24090
10013    27700
10014    31959
10016    54183
10017    16575
10018     5229
10019    42870
10020        0
10021    43631
10022    31924
10023    60998
10024    59283
10025    94600
10026    34003
10027    59707
10028    45141
10029    76003
         ...  
11385    98592
11411    18556
11412    34882
11413    38912
11414    26148
11415    19341
11416    24861
11417    28967
11418    36256
11419    47211
11420    44354
11421    39127
11422    30425
11423    29987
11426    17590
11427    23593
11428    19168
11429    25105
11430      184
11432    60809
11433    32687
11434    59129
11435    53687
11436    17949
11691    60035
11692    18540
11693    11916
11694    20408
11697     4079
12345        0
Name: population, Length: 222, dtype: int64

### Fixing "For One Address"

In [88]:
complaints_df = pd.read_sql("SELECT * FROM complaints", conn, index_col = "id")

In [93]:
complaints_df.head(3)

,type,status,method,date,agency_name,location_id,borough_name
id,,,,,,,
1,Loud Music/Party,Closed,ONLINE,02/17/2018,NYPD,1,MANHATTAN
2,Loud Music/Party,Closed,ONLINE,02/17/2018,NYPD,2,BROOKLYN
3,Loud Music/Party,Closed,PHONE,02/17/2018,NYPD,3,BRONX


In [89]:
complaints_df.replace(to_replace = "For One Address", value = "Tree Request", inplace=True)

In [90]:
complaints_df.columns

Index(['type', 'status', 'method', 'date', 'agency_name', 'location_id',
       'borough_name'],
      dtype='object')

In [96]:
complaints_df.groupby('location_id').count()

,type,status,method,date,agency_name,borough_name
location_id,,,,,,
1,7672,7672,7672,7672,7672,7672
2,2753,2753,2753,2753,2753,2753
3,2947,2947,2947,2947,2947,2947
4,10948,10948,10948,10948,10948,10948
5,7983,7983,7983,7983,7983,7983
6,6310,6310,6310,6310,6310,6310
7,7590,7590,7590,7590,7590,7590
8,10920,10920,10920,10920,10920,10920
9,10215,10215,10215,10215,10215,10215


In [92]:
complaints_df.to_sql("complaints", conn, if_exists = 'replace', index_label = "id")

In [ ]:
conn.close()